In [166]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate, count, demographic_parity_difference
from ipynb.fs.defs import ERED

### Load Data

In [167]:
# Overall data
train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
# train_data = pd.read_pickle(open("tfidf/train_tfidf.pkl", "rb"))
# dev_data = pd.read_pickle(open("tfidf/dev_tfidf.pkl", "rb"))

train_X = list(train_data['TFIDF'])
train_y = (train_data['Sentiment'] == 'positive') * 1

dev_X = list(dev_data['TFIDF'])
dev_y  = (dev_data['Sentiment'] == 'positive') * 1

# AAE
# Filter and extract all AAE data
AAE_train_data = train_data[train_data['Demographic'] == 'AAE']
AAE_train_X = list(AAE_train_data['TFIDF'])
AAE_train_y = (AAE_train_data['Sentiment'] == 'positive') * 1

AAE_dev_data = dev_data[dev_data['Demographic'] == 'AAE']
AAE_dev_X = list(AAE_dev_data['TFIDF'])
AAE_dev_y  = (AAE_dev_data['Sentiment'] == 'positive') * 1

# SAE
# Filter and extract all SAE data
SAE_train_data = train_data[train_data['Demographic'] == 'SAE']
SAE_train_X = list(SAE_train_data['TFIDF'])
SAE_train_y = (SAE_train_data['Sentiment'] == 'positive') * 1

SAE_dev_data = dev_data[dev_data['Demographic'] == 'SAE']
SAE_dev_X = list(SAE_dev_data['TFIDF'])
SAE_dev_y  = (SAE_dev_data['Sentiment'] == 'positive') * 1

---

### Train Logistic Regression Model

In [168]:
lr = LogisticRegression(max_iter=500)
lr.fit(train_X, train_y)

LogisticRegression(max_iter=500)

### Prediction

#### 1. Overall

In [169]:
overall_labels_predict = lr.predict(dev_X)
overall_confusion = confusion_matrix(dev_y, overall_labels_predict)
print(classification_report(dev_y, overall_labels_predict))
print(overall_confusion)

              precision    recall  f1-score   support

           0       0.71      0.68      0.69      2000
           1       0.69      0.72      0.70      2000

    accuracy                           0.70      4000
   macro avg       0.70      0.70      0.70      4000
weighted avg       0.70      0.70      0.70      4000

[[1358  642]
 [ 565 1435]]


#### 2. AAE

In [170]:
AAE_labels_predict = lr.predict(AAE_dev_X)
AAE_confusion = confusion_matrix(AAE_dev_y, AAE_labels_predict)
print(classification_report(AAE_dev_y, AAE_labels_predict))
print(AAE_confusion)

              precision    recall  f1-score   support

           0       0.69      0.60      0.64      1000
           1       0.65      0.73      0.69      1000

    accuracy                           0.66      2000
   macro avg       0.67      0.66      0.66      2000
weighted avg       0.67      0.66      0.66      2000

[[599 401]
 [270 730]]


#### 3. SAE

In [171]:
SAE_labels_predict = lr.predict(SAE_dev_X)
SAE_confusion = confusion_matrix(SAE_dev_y, SAE_labels_predict)
print(classification_report(SAE_dev_y, SAE_labels_predict))
print(SAE_confusion)

              precision    recall  f1-score   support

           0       0.72      0.76      0.74      1000
           1       0.75      0.70      0.72      1000

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

[[759 241]
 [295 705]]


### Bias Evaluation

#### 1. Accuracy + Demographic count

In [172]:
multi_metrics = {'accuracy': accuracy_score, 'count': count}
am = MetricFrame(metrics=multi_metrics, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(am.overall)
print(am.by_group)

accuracy    0.69825
count          4000
dtype: object
            accuracy count
Demographic               
AAE           0.6645  2000
SAE            0.732  2000


#### 2. Selection rate + Demographic parity difference

In [173]:
sm = MetricFrame(metrics=selection_rate, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(sm.overall)
print(sm.by_group)

group_metrics = demographic_parity_difference(y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print("\nDifference between selection rate")
print("Demographic parity difference: {}".format(round(group_metrics, 3)))

0.51925
Demographic
AAE    0.5655
SAE     0.473
Name: selection_rate, dtype: object

Difference between selection rate
Demographic parity difference: 0.093


#### 3. Error Rate Equality Difference

In [174]:
matrix_lst = [AAE_confusion, SAE_confusion]
ERED.cal_sum_ered(overall_confusion, matrix_lst)

Sum of FPR difference: 0.16
Sum of FNR difference: 0.025
Sum of Error Rate Equality Difference: 0.185


0.185

---

## Bias Mitigation

### Exponentiated Gradient

In [175]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
np.random.seed(0)

In [176]:
constraint = DemographicParity()
classifier = LogisticRegression(max_iter=500)
mitigator = ExponentiatedGradient(classifier, constraint)
mitigator.fit(np.array(train_X), train_y, sensitive_features=train_data['Demographic'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x14a145930>,
                      estimator=LogisticRegression(max_iter=500),
                      nu=0.0011462573694523957)

### Prediction

#### 1. Overall

In [177]:
overall_labels_predict = mitigator.predict(dev_X)
overall_confusion = confusion_matrix(dev_y, overall_labels_predict)
print(classification_report(dev_y, overall_labels_predict))
print(overall_confusion)

              precision    recall  f1-score   support

           0       0.70      0.68      0.69      2000
           1       0.69      0.70      0.70      2000

    accuracy                           0.69      4000
   macro avg       0.69      0.69      0.69      4000
weighted avg       0.69      0.69      0.69      4000

[[1362  638]
 [ 590 1410]]


#### 2. AAE

In [178]:
AAE_labels_predict = mitigator.predict(AAE_dev_X)
AAE_confusion = confusion_matrix(AAE_dev_y, AAE_labels_predict)
print(classification_report(AAE_dev_y, AAE_labels_predict))
print(AAE_confusion)

              precision    recall  f1-score   support

           0       0.67      0.64      0.65      1000
           1       0.65      0.69      0.67      1000

    accuracy                           0.66      2000
   macro avg       0.66      0.66      0.66      2000
weighted avg       0.66      0.66      0.66      2000

[[639 361]
 [315 685]]


#### 3. SAE

In [179]:
SAE_labels_predict = mitigator.predict(SAE_dev_X)
SAE_confusion = confusion_matrix(SAE_dev_y, SAE_labels_predict)
print(classification_report(SAE_dev_y, SAE_labels_predict))
print(SAE_confusion)

              precision    recall  f1-score   support

           0       0.73      0.72      0.72      1000
           1       0.72      0.73      0.73      1000

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

[[722 278]
 [271 729]]


### Bias Evaluation

#### 1. Accuracy + Demographic count

In [180]:
multi_metrics = {'accuracy': accuracy_score, 'count': count}
am = MetricFrame(metrics=multi_metrics, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(am.overall)
print(am.by_group)

accuracy    0.693
count        4000
dtype: object
            accuracy count
Demographic               
AAE           0.6575  2000
SAE           0.7285  2000


#### 2. Selection rate + Demographic parity difference

In [181]:
sm = MetricFrame(metrics=selection_rate, y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print(sm.overall)
print(sm.by_group)

group_metrics = demographic_parity_difference(y_true=dev_y, y_pred=overall_labels_predict, sensitive_features=dev_data['Demographic'])
print("\nDifference between selection rate")
print("Demographic parity difference: {}".format(round(group_metrics, 3)))

0.512
Demographic
AAE    0.5235
SAE    0.5005
Name: selection_rate, dtype: object

Difference between selection rate
Demographic parity difference: 0.023


#### 3. Error Rate Equality Difference

In [182]:
matrix_lst = [AAE_confusion, SAE_confusion]
ERED.cal_sum_ered(overall_confusion, matrix_lst)

Sum of FPR difference: 0.083
Sum of FNR difference: 0.044
Sum of Error Rate Equality Difference: 0.127


0.12699999999999995